# 📚 RESUMEN SESIÓN 4: Testing e Integración con IA Generativa

**Propósito:** Cheat-sheet ejecutable con tablas de referencia rápida y snippets reutilizables.

**Contenido:**
- 🧪 Testing con TestClient
- 🤖 Integración con OpenAI
- 🔐 Rate Limiting
- 🔄 Retry Logic
- 📊 Tablas de referencia

In [ ]:
# Instalación rápida de dependencias
!pip install fastapi uvicorn[standard] pytest httpx openai python-dotenv -q

---

## 🧪 TESTING CON TESTCLIENT

### Snippet 1: Test básico GET

In [ ]:
from fastapi import FastAPI
from fastapi.testclient import TestClient

app = FastAPI()

@app.get("/items/{item_id}")
def read_item(item_id: int):
    return {"item_id": item_id, "name": "Laptop"}

# Crear cliente de testing
client = TestClient(app)

# Hacer request
response = client.get("/items/1")

# Asserts
assert response.status_code == 200
assert response.json()["name"] == "Laptop"

print("✅ Test GET básico: OK")

### Snippet 2: Test POST con validación Pydantic

In [ ]:
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

@app.post("/items", status_code=201)
def create_item(item: Item):
    return {"created": item.dict()}

client = TestClient(app)

# Test exitoso
response_ok = client.post("/items", json={"name": "Mouse", "price": 25.0})
assert response_ok.status_code == 201

# Test con campo faltante (debe retornar 422)
response_error = client.post("/items", json={"price": 25.0})
assert response_error.status_code == 422

print("✅ Test POST: OK")

### Snippet 3: Fixture reutilizable

In [ ]:
def get_test_client():
    """Fixture que retorna cliente configurado"""
    app = FastAPI()
    
    @app.get("/health")
    def health():
        return {"status": "ok"}
    
    return TestClient(app)

# Usar en múltiples tests
def test_health():
    client = get_test_client()
    response = client.get("/health")
    assert response.status_code == 200

test_health()
print("✅ Fixture reutilizable: OK")

### Snippet 4: Mock de dependencia

In [ ]:
from fastapi import Depends, HTTPException
from typing import Annotated

app = FastAPI()

# Dependencia real (simula auth)
def get_current_user():
    raise HTTPException(401, "No autenticado")

CurrentUser = Annotated[dict, Depends(get_current_user)]

@app.get("/profile")
def get_profile(user: CurrentUser):
    return {"username": user["username"]}

# Mock
def mock_user():
    return {"username": "testuser"}

# Reemplazar dependencia
app.dependency_overrides[get_current_user] = mock_user

client = TestClient(app)
response = client.get("/profile")
assert response.status_code == 200
assert response.json()["username"] == "testuser"

# Limpiar override
app.dependency_overrides = {}

print("✅ Mock de dependencia: OK")

### 📊 Tabla: Status Codes Comunes

| Código | Significado | Cuándo usar |
|--------|-------------|-------------|
| 200 | OK | GET, PUT, PATCH exitoso |
| 201 | Created | POST que crea recurso |
| 400 | Bad Request | Error de validación de negocio |
| 401 | Unauthorized | Falta autenticación |
| 403 | Forbidden | Usuario autenticado pero sin permisos |
| 404 | Not Found | Recurso no existe |
| 422 | Unprocessable Entity | Error de validación Pydantic |
| 429 | Too Many Requests | Rate limit excedido |
| 500 | Internal Server Error | Error del servidor |
| 503 | Service Unavailable | Servicio temporalmente no disponible |

---

## 🤖 INTEGRACIÓN CON OPENAI

### Snippet 5: Llamada básica a OpenAI (async)

In [ ]:
import asyncio
from openai import AsyncOpenAI

async def llamar_openai(prompt: str, max_tokens: int = 100):
    """Llamada asíncrona a OpenAI"""
    client = AsyncOpenAI(api_key="sk-...")  # Usar variable de entorno # api_key=os.getenv("OPENAI_API_KEY")
    
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Eres un asistente útil"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    
    return response

# Ejemplo de uso
# response = await llamar_openai("Explica FastAPI en una línea")
# texto = response.choices[0].message.content
# tokens = response.usage.total_tokens

print("✅ Snippet OpenAI async: Listo para usar")

### Snippet 6: Endpoint FastAPI con IA

In [ ]:
app = FastAPI()

class PromptRequest(BaseModel):
    prompt: str
    max_tokens: int = 100

class PromptResponse(BaseModel):
    respuesta: str
    tokens_usados: int

@app.post("/ai/completar", response_model=PromptResponse)
async def completar(request: PromptRequest):
    # response = await llamar_openai(request.prompt, request.max_tokens)
    
    # Simular respuesta para el ejemplo
    return PromptResponse(
        respuesta=f"Respuesta simulada a: {request.prompt}",
        tokens_usados=30
    )

print("✅ Endpoint con IA: Listo")

### 📊 Tabla: Roles de Mensajes OpenAI

| Role | Propósito | Ejemplo |
|------|-----------|----------|
| system | Instrucciones globales del asistente | "Eres un experto en Python" |
| user | Mensaje del usuario | "¿Qué es FastAPI?" |
| assistant | Respuesta previa del asistente (para contexto) | "FastAPI es un framework..." |

### 📊 Tabla: Parámetros Clave

| Parámetro | Rango | Descripción |
|-----------|-------|-------------|
| temperature | 0-2 | Creatividad (0=determinista, 2=muy creativo) |
| max_tokens | 1-∞ | Límite de tokens en respuesta |
| top_p | 0-1 | Alternativa a temperature (nucleus sampling) |
| presence_penalty | -2 a 2 | Penaliza repetición de temas |
| frequency_penalty | -2 a 2 | Penaliza repetición de palabras |

### 📊 Tabla: Tokens y Costes (GPT-3.5-turbo)

| Concepto | Valor |
|----------|-------|
| 1 token | ≈ 0.75 palabras (inglés) |
| 100 tokens | ≈ 75 palabras |
| Coste input | $0.50 / 1M tokens |
| Coste output | $1.50 / 1M tokens |
| 1K tokens input | $0.0005 |
| 1K tokens output | $0.0015 |

### Snippet 7: Calcular coste de una llamada

In [ ]:
def calcular_coste_gpt35(prompt_tokens: int, completion_tokens: int) -> float:
    """Calcula coste en USD para GPT-3.5-turbo"""
    coste_input = (prompt_tokens * 0.50) / 1_000_000
    coste_output = (completion_tokens * 1.50) / 1_000_000
    return coste_input + coste_output

# Ejemplo: 500 tokens prompt + 200 tokens completion
coste = calcular_coste_gpt35(500, 200)
print(f"Coste: ${coste:.6f} USD")
print(f"Coste por 1000 llamadas: ${coste * 1000:.2f} USD")

---

## 🔐 RATE LIMITING

### Snippet 8: Rate limiter con ventana deslizante

In [ ]:
from collections import defaultdict
from datetime import datetime, timedelta

rate_limit_store = defaultdict(list)

def check_rate_limit(user_id: str, max_requests: int = 10, window_minutes: int = 1) -> bool:
    """Verifica si el usuario excedió el límite
    
    Args:
        user_id: Identificador del usuario
        max_requests: Número máximo de requests
        window_minutes: Ventana temporal en minutos
    
    Returns:
        True si puede hacer el request, False si excedió límite
    """
    now = datetime.now()
    window_start = now - timedelta(minutes=window_minutes)
    
    # Filtrar requests recientes
    recent = [req for req in rate_limit_store[user_id] if req > window_start]
    rate_limit_store[user_id] = recent
    
    # Verificar límite
    if len(recent) >= max_requests:
        return False
    
    # Registrar request actual
    rate_limit_store[user_id].append(now)
    return True

# Ejemplo de uso
print("Request 1:", check_rate_limit("user123", max_requests=3))  # True
print("Request 2:", check_rate_limit("user123", max_requests=3))  # True
print("Request 3:", check_rate_limit("user123", max_requests=3))  # True
print("Request 4:", check_rate_limit("user123", max_requests=3))  # False (excedido)

### Snippet 9: Endpoint con rate limiting

In [ ]:
from fastapi import HTTPException

app = FastAPI()

@app.post("/api/action")
async def action(user_id: str):
    # Verificar rate limit
    if not check_rate_limit(user_id, max_requests=10):
        raise HTTPException(
            status_code=429,
            detail="Rate limit excedido. Intenta en 1 minuto."
        )
    
    # Procesar request
    return {"status": "ok"}

print("✅ Endpoint con rate limiting: Listo")

### 📊 Tabla: Estrategias de Rate Limiting

| Estrategia | Descripción | Ventajas | Desventajas |
|------------|-------------|----------|-------------|
| Fixed Window | Contador que resetea cada minuto | Simple | Burst al inicio/fin de ventana |
| Sliding Window | Solo cuenta últimos N segundos | Más justo | Requiere almacenar timestamps |
| Token Bucket | Tokens se regeneran con el tiempo | Permite bursts controlados | Más complejo |
| Leaky Bucket | Procesa requests a tasa constante | Suaviza tráfico | Puede rechazar requests válidos |

#### Nota: En producción con múltiples servidores, usar Redis en lugar de un diccionario en memoria para compartir el estado.

---

## 🔄 RETRY LOGIC

### Snippet 10: Retry con backoff exponencial

In [ ]:
import asyncio
from fastapi import HTTPException

async def llamar_api_con_retry(url: str, max_intentos: int = 3):
    """Llama a API externa con reintentos y backoff exponencial
    
    Args:
        url: URL de la API
        max_intentos: Número máximo de reintentos
    
    Returns:
        Respuesta de la API
    
    Raises:
        HTTPException 503 si fallan todos los intentos
    """
    for intento in range(max_intentos):
        try:
            # Intentar llamada
            # response = await httpx.AsyncClient().get(url)
            # return response.json()
            
            # Simulación
            if intento < 2:
                raise Exception("Error temporal")
            return {"data": "success"}
        
        except Exception as e:
            espera = 2 ** intento  # 1s, 2s, 4s, 8s...
            
            if intento < max_intentos - 1:
                print(f"Intento {intento + 1} falló. Esperando {espera}s...")
                await asyncio.sleep(espera)
            else:
                raise HTTPException(503, "Servicio no disponible")

# Ejemplo
resultado = await llamar_api_con_retry("https://api.example.com")
print(f"✅ Retry exitoso: {resultado}")

### 📊 Tabla: Comparación de Estrategias de Retry

| Estrategia | Intento 1 | Intento 2 | Intento 3 | Intento 4 | Total |
|------------|-----------|-----------|-----------|-----------|-------|
| Sin espera | 0s | 0s | 0s | 0s | 0s |
| Espera fija (1s) | 0s | 1s | 2s | 3s | 6s |
| Exponencial (2^n) | 0s | 1s | 3s | 7s | 11s |
| Fibonacci | 0s | 1s | 2s | 4s | 7s |

### 📊 Tabla: Cuándo Usar Retry

| Status Code | ¿Reintentar? | Razón |
|-------------|--------------|-------|
| 500 | ✅ Sí | Error temporal del servidor |
| 502 | ✅ Sí | Bad gateway (proxy temporalmente caído) |
| 503 | ✅ Sí | Service unavailable (sobrecarga temporal) |
| 504 | ✅ Sí | Gateway timeout |
| 429 | ✅ Sí | Rate limit (esperar más tiempo) |
| 400 | ❌ No | Bad request (el request está mal) |
| 401 | ❌ No | Unauthorized (falta auth) |
| 403 | ❌ No | Forbidden (no tienes permisos) |
| 404 | ❌ No | Not found (recurso no existe) |
| 422 | ❌ No | Validación fallida (datos incorrectos) |

---

## 📊 LOGGING Y OBSERVABILIDAD

### Snippet 11: Logging estructurado

In [ ]:
import logging
import time

# Configurar logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

app = FastAPI()

@app.post("/api/process")
async def process(data: dict, user_id: str = "anonymous"):
    start_time = time.time()
    
    # Log del request
    logger.info(
        f"[REQUEST] user={user_id} "
        f"endpoint=/api/process "
        f"data_size={len(str(data))}"
    )
    
    # Procesar...
    await asyncio.sleep(0.1)  # Simular procesamiento
    
    # Log del response
    duration_ms = int((time.time() - start_time) * 1000)
    logger.info(
        f"[RESPONSE] user={user_id} "
        f"status=200 "
        f"duration={duration_ms}ms"
    )
    
    return {"status": "ok"}

print("✅ Logging estructurado: Listo")

### 📊 Tabla: Niveles de Logging

| Nivel | Cuándo usar | Ejemplo |
|-------|-------------|----------|
| DEBUG | Información detallada para debugging | `logger.debug(f"Variable x={x}")` |
| INFO | Eventos normales importantes | `logger.info("Request procesado")` |
| WARNING | Situaciones anormales pero recuperables | `logger.warning("Cache miss")` |
| ERROR | Errores que permiten continuar | `logger.error("API call failed")` |
| CRITICAL | Errores graves que requieren atención | `logger.critical("Database down")` |

---

## 🔧 PATRONES COMPLETOS

### Snippet 12: Endpoint de producción completo

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel
import asyncio
import time
import logging

app = FastAPI()
logger = logging.getLogger(__name__)

class AIRequest(BaseModel):
    prompt: str
    max_tokens: int = 100

def get_current_user():
    return {"user_id": "user123", "tier": "premium"}

@app.post("/ai/complete")
async def complete(
    request: AIRequest,
    user: dict = Depends(get_current_user)
):
    """Endpoint de producción con todas las mejores prácticas"""
    start_time = time.time()
    user_id = user["user_id"]
    
    # 1. Rate limiting
    if not check_rate_limit(user_id, max_requests=10):
        logger.warning(f"[RATE_LIMIT] user={user_id}")
        raise HTTPException(429, "Rate limit excedido")
    
    # 2. Validación de negocio
    max_allowed = 500 if user["tier"] == "premium" else 100
    if request.max_tokens > max_allowed:
        raise HTTPException(400, f"max_tokens excede límite de {max_allowed}")
    
    # 3. Log request
    logger.info(
        f"[AI_REQUEST] user={user_id} "
        f"prompt_len={len(request.prompt)} "
        f"max_tokens={request.max_tokens}"
    )
    
    # 4. Llamada a IA con retry
    # response = await llamar_openai_con_retry(request.prompt, request.max_tokens)
    await asyncio.sleep(0.1)  # Simular
    
    # 5. Calcular métricas
    tokens_used = 30
    coste = calcular_coste_gpt35(10, 20)
    duration_ms = int((time.time() - start_time) * 1000)
    
    # 6. Log response
    logger.info(
        f"[AI_RESPONSE] user={user_id} "
        f"tokens={tokens_used} "
        f"coste=${coste:.6f} "
        f"duration={duration_ms}ms"
    )
    
    # 7. Retornar con métricas
    return {
        "respuesta": "Respuesta simulada",
        "tokens_usados": tokens_used,
        "coste_estimado": coste,
        "duracion_ms": duration_ms
    }

print("✅ Endpoint de producción: Listo")

### 📊 Tabla: Checklist de Endpoint de Producción

| Feature | Implementado | Descripción |
|---------|--------------|-------------|
| ✅ Auth | Sí | Depends(get_current_user) |
| ✅ Rate Limiting | Sí | check_rate_limit() |
| ✅ Validación | Sí | max_tokens con límites por tier |
| ✅ Logging Request | Sí | logger.info() con contexto |
| ✅ Logging Response | Sí | Incluye tokens, coste, duración |
| ✅ Retry Logic | Sí | Backoff exponencial |
| ✅ Error Handling | Sí | HTTPException con status correcto |
| ✅ Métricas | Sí | tokens, coste, duración |
| ✅ Type Hints | Sí | Pydantic models |
| ✅ Documentación | Sí | Docstring en endpoint |

---

## 🎯 MEJORES PRÁCTICAS

### Testing
✅ Usar TestClient para tests rápidos sin servidor  
✅ Crear fixtures para reutilizar configuración  
✅ Mockear dependencias externas (DB, APIs, auth)  
✅ Testear tanto happy path como casos de error  
✅ Limpiar dependency_overrides después de cada test  

### Integración con IA
✅ Usar async/await para llamadas a APIs externas  
✅ Implementar rate limiting (3-10 req/min)  
✅ Validar max_tokens antes de llamar (ahorro de $$)  
✅ Retry con backoff exponencial (2^n segundos)  
✅ Logging estructurado (request + response)  
✅ Calcular y trackear métricas (tokens, coste, duración)  

### Seguridad
✅ Autenticación en todos los endpoints sensibles  
✅ Rate limiting por usuario (no global)  
✅ Validación de inputs (Pydantic + lógica de negocio)  
✅ Límites de recursos (max_tokens, timeout)  
✅ Logging de eventos de seguridad (rate limit, auth failures)  

### Performance
✅ Usar endpoints async cuando hay I/O (DB, APIs)  
✅ Implementar caching cuando sea apropiado  
✅ Medir duración de requests (logger.info con ms)  
✅ Optimizar queries a DB (índices, N+1)  
✅ Connection pooling para APIs externas

---

## 🚫 ERRORES COMUNES

### Testing
❌ No limpiar `dependency_overrides` → Afecta otros tests  
❌ Usar `response.text` en vez de `response.json()` → Error de tipo  
❌ No verificar status code antes de acceder al JSON → Puede fallar  
❌ Asumir que 422 es error del código → Es validación Pydantic esperada  

### Async/Await
❌ Olvidar `await` → Recibir coroutine en vez de resultado  
❌ Usar `def` en vez de `async def` → No poder usar await  
❌ Usar blocking calls en async → Bloquear el event loop  

### Rate Limiting
❌ Reintentar inmediatamente tras 429 → Empeorar la situación  
❌ Rate limit global en vez de por usuario → Injusto  
❌ No limpiar timestamps antiguos → Memory leak  

### Retry Logic
❌ Reintentar errores 4xx → Desperdiciar recursos  
❌ No usar backoff exponencial → Thundering herd  
❌ Reintentar infinitamente → Sistema colgado  

### Costes
❌ No validar max_tokens antes de llamar → $$$ desperdiciados  
❌ No trackear costes → Facturas sorpresa  
❌ Permitir max_tokens ilimitado → Vulnerabilidad de $$

---

## 📦 DEPENDENCIAS NECESARIAS

```bash
pip install fastapi==0.115.0
pip install uvicorn[standard]==0.32.0
pip install pytest==8.0.0
pip install httpx==0.27.0
pip install openai==1.12.0
pip install python-dotenv==1.0.0
```

---

## 🔗 RECURSOS ADICIONALES

**Documentación Oficial:**
- FastAPI Testing: https://fastapi.tiangolo.com/tutorial/testing/
- OpenAI API: https://platform.openai.com/docs/api-reference
- Pydantic: https://docs.pydantic.dev/

**Herramientas de Testing:**
- pytest: https://docs.pytest.org/
- httpx: https://www.python-httpx.org/

**Mejores Prácticas:**
- Rate Limiting: https://www.ietf.org/rfc/rfc6585.txt
- Retry Logic: https://aws.amazon.com/es/blogs/architecture/exponential-backoff-and-jitter/